<a href="https://colab.research.google.com/github/JoeHeeJae/Movie-Recommendation-System-Colon-coding/blob/main/Movies_Latent_Factor_Collaborative_Filtering_Colon_Coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#잠재 요소 협업 필터링
Latent Factor Collaborative Filtering

In [1]:
import pandas as pd
import numpy as np
import sklearn

##데이터 불러오기

In [6]:
rating_data = pd.read_csv('/content/drive/MyDrive/Data_csv/ratings.csv')
movie_data = pd.read_csv('/content/drive/MyDrive/Data_csv/movies.csv')

In [7]:
rating_data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [8]:
movie_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
print(movie_data.shape)
print(rating_data.shape)

(9125, 3)
(100004, 4)


##데이터 전처리

잠재요인 기반 협업 필터링 추천시스템에서는 timestamp와 genres는 필요없다.
drop시키기

In [12]:
rating_data.drop('timestamp', axis=1, inplace=True)
rating_data.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [13]:
movie_data.drop('genres', axis=1, inplace=True)
movie_data.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [14]:
#두 파일 merge
user_movie_data = pd.merge(rating_data, movie_data, on ='movieId')
user_movie_data.head()

,userId,movieId,rating,title
0,1,31,2.5,Dangerous Minds (1995)
1,7,31,3.0,Dangerous Minds (1995)
2,31,31,4.0,Dangerous Minds (1995)
3,32,31,4.0,Dangerous Minds (1995)
4,36,31,3.0,Dangerous Minds (1995)


In [15]:
user_movie_data.shape

(100004, 4)

In [20]:
#colums = movie , row = userId, null = 0
user_movie_rating = user_movie_data.pivot_table('rating', index = 'userId', columns = 'title').fillna(0)

In [24]:
user_movie_rating.shape

(671, 9064)

In [23]:
user_movie_rating.head()

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
#특정 영화와 비슷한 영화를 추천
#user - movie > movie - user 로 Transpose
movie_user_rating = user_movie_rating.values.T
movie_user_rating.shape

(9064, 671)

##SVD(Singular Value Decomposition)
특이값 분해

In [28]:
#sickit learn_TruncatedSVD 사용
from sklearn.decomposition import TruncatedSVD

In [29]:
SVD = TruncatedSVD(n_components=12) # latent = 12
matrix = SVD.fit_transform(movie_user_rating)
matrix.shape

(9064, 12)

In [30]:
matrix[0]

array([ 0.01227491,  0.00250779,  0.01554857, -0.03397652, -0.01447921,
        0.00360659, -0.00223029,  0.04498711, -0.01594867, -0.0211599 ,
        0.01097741, -0.01356719])

In [36]:
#피어슨 상관계수를 통해 데이터 구함
#np.corrcoef()
corr = np.corrcoef(matrix)
corr.shape

(9064, 9064)

In [37]:
# 특정 영화와 관련하여 상관계수가 높은 영화를 뽑아주기
movie_title = user_movie_rating.columns
movie_title_list = list(movie_title)
coffey_hands = movie_title_list.index("Spider-Man (2002)")

In [38]:
corr_coffey_hands = corr[coffey_hands]
list(movie_title[(corr_coffey_hands >= 0.9)])[:50]

['28 Days Later (2002)',
 'A.I. Artificial Intelligence (2001)',
 'American Pie 2 (2001)',
 'Anger Management (2003)',
 'Aviator, The (2004)',
 'Blade II (2002)',
 'Bourne Identity, The (2002)',
 'Bourne Supremacy, The (2004)',
 'Bruce Almighty (2003)',
 "Charlie's Angels (2000)",
 'Collateral (2004)',
 'Fast and the Furious, The (2001)',
 'From Hell (2001)',
 'Gangs of New York (2002)',
 'Gladiator (2000)',
 'Gone in 60 Seconds (2000)',
 'Harry Potter and the Chamber of Secrets (2002)',
 'Harry Potter and the Prisoner of Azkaban (2004)',
 "Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)",
 'Hero (Ying xiong) (2002)',
 'Ice Age (2002)',
 'Ice Age 2: The Meltdown (2006)',
 'Identity (2003)',
 'Italian Job, The (2003)',
 'Kindergarten Cop (1990)',
 'King Kong (2005)',
 'Lara Croft: Tomb Raider (2001)',
 'Lord of the Rings: The Fellowship of the Ring, The (2001)',
 'Lord of the Rings: The Return of the King, The (2003)',
 'Lord of the Rings: 

##결과